In [1]:
import os 
import json
import traceback
import pandas as pd 

In [2]:
from dotenv import load_dotenv
load_dotenv()

# Get openai_key from .env
SECRET_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key=SECRET_KEY,
    model_name="gpt-3.5-turbo",
    temperature=0.7,
    max_tokens=128
    )